# Interpretable Machine Learning with SHAP Values

In [ ]:
import pandas as pd
import shap
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
import numpy as np
import matplotlib.pyplot as plt
from ydata_profiling import ProfileReport

%matplotlib inline



shap.initjs()

np.random.seed(1234)

#selected_features = ['MedInc', 'HouseAge', 'AveRooms', 'AveBedrms', 'Population', 'AveOccup']
X, y = shap.datasets.california()

# combined dataframe
df = pd.DataFrame(X, columns=X.columns)
df['price'] = y

#X = X[selected_features]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [ ]:
# # Generate a profile report

# # Or use the minimal option which might have fewer widget dependencies
# profile = ProfileReport(df, title="California Housing Dataset Profile", minimal=False)
# # Save the report to an HTML file
# profile.to_file("california_housing_report.html")

# A Motivating Example: Linear Regression

Linear regression models are simply enough to be considered "intrinsically interpretable". The prediction from a linear model is simply the sum of the products of model coefficients and the variable values. 

$$
y = \beta_0 + \beta_1 x_1 + \dots + \beta_k x_k
$$

It might be tempting to use the magnitudes of the coefficients to determine the most important variables, but this would provide a misleading understanding of the model if the units of measure of the variables are different. 

In the California housing dataset, median income is measured in tens of thousands of dollars and the coefficient is 0.54, which is a similar magnitude to the rest of the coefficients. However, if we measure median income in dollars, then coefficient would 5 orders of magnitude greater than the rest of the coefficients and would be deemed the "most important" by magnitude. 

In [ ]:
# Train a linear regression model using the specified features
linear_model_features = ['MedInc', 'HouseAge', 'AveRooms', 'AveOccup', 'Population', 'AveBedrms']
X_train_selected = X_train[linear_model_features]
X_test_selected = X_test[linear_model_features]

# Initialize and train the linear regression model
linear_model = LinearRegression()
linear_model.fit(X_train_selected, y_train)

# Print the model coefficients
coefficients = pd.DataFrame({
    'Feature': linear_model_features,
    'Coefficient': linear_model.coef_
})
print("Linear Regression Coefficients:")
print(coefficients)
print(f"\nIntercept: {linear_model.intercept_:.4f}")

There is a way to view the importance of each variable for an *individual prediction*. Since the predicted value of a linear model is a linear sum of coefficients multipled by the value of the variable (the $\beta_i  x_i$ terms in the sum), we can decompose the predictions into these products and view their magnitudes since they are all in the same unit of measure as the target variable.

In [ ]:
# Calculate predictions manually by multiplying features by coefficients and adding intercept
manual_predictions = np.dot(X_test_selected, linear_model.coef_) + linear_model.intercept_
np.allclose(manual_predictions, linear_model.predict(X_test_selected))

In [ ]:
# create a matrix of the products
linear_components = pd.DataFrame(np.concatenate(
    [
        np.repeat(linear_model.intercept_, X_test_selected.shape[0]).reshape((-1, 1)), 
        np.multiply(X_test_selected.values, linear_model.coef_)
    ], 
axis = 1), 
    columns = ['Intercept'] + linear_model_features
)

np.allclose(linear_components.sum(axis=1), (manual_predictions))

We can use these products as feature importances to create some simple visualizations such as a waterfall plot that shows how to get from the intercept of the model to the predicted value. 

In [ ]:
from utils import plot_linear_prediction_waterfall
plot_linear_prediction_waterfall(3, linear_components=linear_components,model=linear_model, X_test_selected=X_test_selected)

# Model Explainability for Tree-Based Models

We just saw how linear models are easily interpreted and how you can get a nice, clean, preduction 

In modern applications, the best results are often obtained using large datasets and complex, non-linear models whose output is difficult to interpret and whose inner mechanics are non-transparent and difficult to understand. In many settings, tree-based methods such as random forests and gradient boosting machines achieve the best performance, especially on tabular data that is often found in the business world. 

(insert shap citation) identify three desirable properties that a good feature importance measure should have

1. Consistency: If the model changes in such a way as to increase the marginal importance of a feature, the feature importance for that feature should not decrease
2. Prediction-level explainations: the feature importance measure can explain individual predictions, not just the global importance for the entire model
3. Local Accuracy (Additivity): the sum of the feature importance measures for an individual prediction sum to the predicted value, i.e. $f(x) = \sum_{i} \phi_i$ where the $\phi_i$ are the feature importance measures for the $i$ th feature. (Note that this requires prediction-level explainations)


There are numerous feature importance measures for use with both trees and other classes of models. The ones that are the most often used are:

1. Permutation Importance 
2. Gain

We will give an example of each and explain why none of these satisfy all three properties and then introduce SHAP as the only feature importance measure that satisfied all three. 



In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error


# Set up hyperparameter search for Random Forest Regressor
# Define the parameter grid to search
param_dist = {
    'n_estimators': [50, 100, 150, 200],
    'max_depth': [None, 10, 20, 30, 40],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['sqrt', 'log2'],
    'bootstrap': [True, False]
}

# Create the random forest regressor
rf = RandomForestRegressor(random_state=42)

# Set up k-fold cross-validation
cv = KFold(n_splits=5, shuffle=True, random_state=42)

# Create the RandomizedSearchCV
rf_random = RandomizedSearchCV(
    estimator=rf,
    param_distributions=param_dist,
    n_iter=50,  # Number of parameter settings to try
    scoring='neg_mean_squared_error',
    cv=cv,
    verbose=1,
    random_state=42,
    n_jobs=-1  # Use all available cores
)

# Fit the random search
rf_random.fit(X_train, y_train)

# Print the best parameters and score
print("Best Parameters:", rf_random.best_params_)
print("Best CV Score:", -rf_random.best_score_)  # Convert back to positive MSE

# Evaluate on test set
best_rf = rf_random.best_estimator_
test_score = best_rf.score(X_test, y_test)
print(f"R² Score on Test Set: {test_score:.4f}")

# Calculate MSE on test set
y_pred = best_rf.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
print(f"MSE on Test Set: {mse:.4f}")

## Permutation Importance

Permutation importance works by randomly shuffing a feature then watching how accuracy of the model degrades. By breaking the dependence between the feature and the target variable, the idea is that we can see how much the model truely relies on that feature.

Since permutation importances is measured with respect to model performance, computing it using the training set can provide misleading results on overfitted models, so it is best practice to calculate it using data not used to train the model. 


In [ ]:
from sklearn.inspection import permutation_importance

# Calculate permutation importance for RandomForest model

# Calculate permutation importance
perm_importance = permutation_importance(best_rf, X_test, y_test, n_repeats=50, random_state=42)

# Sort the permutation importance by value
sorted_indices = perm_importance.importances_mean.argsort()
sorted_importances = perm_importance.importances_mean[sorted_indices]
sorted_features = X.columns[sorted_indices]
sorted_std = perm_importance.importances_std[sorted_indices]

# Create horizontal bar plot
plt.figure(figsize=(10, 6))
plt.barh(range(len(sorted_features)), sorted_importances, xerr=sorted_std, 
         color='skyblue', edgecolor='navy', alpha=0.7)

# Add feature names as y-tick labels
plt.yticks(range(len(sorted_features)), sorted_features)

# Add labels and title
plt.xlabel('Permutation Importance (Mean Decrease in Model Performance)')
plt.title('Feature Importance based on Permutation Importance')

# Add grid for better readability
plt.grid(axis='x', linestyle='--', alpha=0.6)

# Adjust layout and display
plt.tight_layout()
plt.show()

# Print the actual values for reference
for feature, importance, std in zip(sorted_features, sorted_importances, sorted_std):
    print(f"{feature}: {importance:.4f} ± {std:.4f}")

### Advantages

* Easy to understand and implement; provides a quick, global overview of feature importance
* Can be used with any model
* Satisfies the *consistency* property

### Disadvantages

* Permuting the values of a single features can produce data points outside the distribution of the data when features are correlated. 
* Does not provide prediction-level feature importances and so cannot be locally accurate
* Permutation importance can split the importance between correlated features, making one (or both) features seem less important than they actually are
* Permutation importance uses the model *performance* instead of *output*, and this may not be what you want depending on the context

The plot below uses some dummy data to illustrate the effect on the data disbritution of permuting a feature when two features are correlated. The points in red are the result of permuting Feature 1, which generates unrealistic data points that are outside the joint distribution of Features 1 and 2. 

In [ ]:
np.random.seed(42)

n_samples = 1000
x1 = np.random.normal(0, 1, n_samples)
x2 = 0.8 * x1 + 0.2 * np.random.normal(0, 1, n_samples)

df_sample = pd.DataFrame({'feature1': x1, 'feature2': x2})

X_permuted = df_sample.copy()

# Select a slice from the middle of feature1's range to permute
lower_bound = np.percentile(df_sample['feature1'], 25)
upper_bound = np.percentile(df_sample['feature1'], 75)
mask = (X_permuted['feature1'] >= lower_bound) & (X_permuted['feature1'] <= upper_bound)
indices_to_permute = X_permuted[mask].index

# Save original values for visualization
original_values = X_permuted.loc[indices_to_permute, 'feature1'].copy()

# Permute feature1 values within the selected range
X_permuted.loc[indices_to_permute, 'feature1'] = np.random.permutation(original_values)

# Calculate correlation
original_corr = df_sample['feature1'].corr(df_sample['feature2'])
permuted_corr = X_permuted['feature1'].corr(X_permuted['feature2'])

# Create scatter plot to visualize the effect of permutation
plt.figure(figsize=(8, 8))

# Plot all points in the original data
plt.scatter(df_sample['feature1'], df_sample['feature2'], 
            alpha=0.5, label='Original Points', color='blue')

# Highlight the permuted points
plt.scatter(X_permuted.loc[indices_to_permute, 'feature1'], 
            X_permuted.loc[indices_to_permute, 'feature2'], 
            alpha=0.7, label='Permuted Points', color='red')

# Add reference lines for the slice boundaries
plt.axvline(x=lower_bound, color='gray', linestyle='--', alpha=0.5)
plt.axvline(x=upper_bound, color='gray', linestyle='--', alpha=0.5)

# Add labels and title
plt.xlabel('Feature 1')
plt.ylabel('Feature 2')
plt.title('Demonstration of Permutation Method Issues with Correlated Features\n'
          f'Original Correlation: {original_corr:.3f}, After Permutation: {permuted_corr:.3f}')
plt.legend()
plt.grid(alpha=0.3)

# Add annotation explaining the problem
plt.annotate(
    "Permutation creates\npoints outside the\njoint distribution",
    xy=(-0.5, 1.5), xytext=(0.5, 2.0),
    arrowprops=dict(facecolor='black', shrink=0.05, width=1.5, headwidth=8),
    fontsize=12,
    bbox=dict(boxstyle="round,pad=0.5", fc="yellow", alpha=0.8)
)

plt.tight_layout()
plt.show()

## Gain 

Gain a feature importance measure that is unique to tree-based models that is calculated as the total reduction of loss or impurity contributed by all splits
for a given feature. 

### Advantages

* Is calculated "for free" with the training of the model


### Disadvantages

* Since it is based on the *training data*, it is suseptible to overfitting
* Does not satisfy any of the three properties of consistency, prediction-level explainations or local accuracy
* It favors continues features over categorical features since there are more opprotunities for splitting. This is also true for high-cardinality categorical features as well that have large numbers of possible splits to choose from
* Not model agnostic, only works with tree-based methods

In [ ]:
# Calculate gain importance from the random forest model
gain_importance = pd.Series(best_rf.feature_importances_, index=X_train.columns).sort_values(ascending=True)

# Create horizontal bar plot for gain importance
plt.figure(figsize=(10, 6))
gain_importance.plot.barh(color='salmon', edgecolor='darkred')

# Add labels and title
plt.xlabel('Gain Importance')
plt.title('Feature Importance based on Gain (Gini Impurity Reduction)')

# Add grid for better readability
plt.grid(axis='x', linestyle='--', alpha=0.6)

# Show the plot
plt.tight_layout()
plt.show()

# Print the actual values
for feature, importance in gain_importance.items():
    print(f"{feature}: {importance:.6f}")

## Introduction to SHAP Values for Tree-Based Models

SHAP values stand for **SHapley Additive Predictions** were introduced in 2017 by Lundberg and Lee [1]. They are based on Shapely values from cooperative game theory, which is a theoretically sound way to fairly allocate the payouts to players in a coopoerative game. We won't take the game theory connections too far here, but you can think of the "game" as the machine learning model being explained, the "players" as the input features to the model, and the "payout" the model predictions. SHAP values calculate the contribution each feature made to the prediction. 

Lundberg and Lee showed that SHAP values are the only explainatory model that satisfies the three properties that we discussed earlier. 

1. Consistency
2. Local Accuracy / Additivity
3. Prediction-level explainations

Formally, additivity means that if $\phi_i$ is the SHAP value for the $i$ feature, the sum of the SHAP values equals *the difference between the model output and the expected output*. 

$$
f(x) = E[f(x)] + \sum_{i=1}^F \phi_i
$$

These properties unlock a variety of rich visualizations and diagnostic plots that we can use in place of the global feature importance measures that we just discussed. They can also be augmented by traditional Partial Dependence Plots and Individual Conditional Expectation plots, both of which we will review later in this presentation. 

### Some SHAP Theory

Formally, let 

$$
\phi_i =  \sum_{S\subseteq F/\{i\}} \frac{1}{|F|}  \frac{1}{\binom{|F|-1}{|S|}} \big[ f_{S\cup\{i\}} (x_{S\cup\{i\}}) - f_S(x_S) \big]
$$

where 

* $|F|$ is the number of features
* $|S|$ is the number of features in the subset $S \subset F$
* $f_{S\cup\{i\}}= E[f(x) | x_{S\cup\{i\}}]$ is the conditional expectation of the model given the features $x_{S\cup\{i\}}$
* $ f_S = E[f(x) | x_S]$ is the conditional expectation of the model given the features $x_S$


$\phi_0 = f_\empty(\empty)$ 

**In Words**

A one-line definition:

> A SHAP value is the average marginal change in the model output from adding a feature to a subset of features, averaged over all such subsets not containing that feature. 

More precisely: 

>To calculate the SHAP value for feature $i$, we consider all possible subsets $S$ of features that exclude feature $i$. For each subset, we compute the marginal contribution of adding feature $i$ to that subset, which is the difference between the expected model output when $S$ and feature $i$ are known versus when only $S$ is known. We then take the weighted average of these contributions across all possible subsets where the weights are related to the number of such subsets. 

In steps: 

1. Train a machine learning model $f$
2. For each feature $i$, consider all the subsets that exclude $i$
3. Compute the expected difference in the expected model outputs $E[f(x) | x_{S\cup\{i\}}] - E[f(x) | x_S]$ with and without the feature
4. Average over all subsets, with weights equal to the probability of selecting that particular subset

Let's unpack the last of these. The weights in the sum above are the probability of selecting a particular subset. The term has two parts: 

Given a feature $i$, the number of subsets of $S\subseteq F / \{i \}$ is
$$
\binom{|F|-1}{|S|}
$$, 

so the probability of selecting a subset, conditional on $i$ (and assuming selection happens uniformly) is

$$
 \frac{1}{\binom{|F|-1}{|S|}} 
$$

Since there are $|F|$ features in the model, the probability of selecting one of them uniformly is $\frac{1}{|F|}$. So the joint probability of selecting feature $i$ and subset $S\subseteq F / \{i \}$ is therefore

$$
\frac{1}{|F|}  \frac{1}{\binom{|F|-1}{|S|}} 
$$




In [ ]:
from math import comb

F = len(X.columns)

combinations = {}
for i in range(1, F):
    combinations[i] = comb(F, i)

# Create a DataFrame to store the combinations
combinations_df = pd.DataFrame.from_dict(combinations, orient='index', columns=['Combinations'])
combinations_df.reset_index(inplace=True)

total_combinations = combinations_df['Combinations'].sum()


In [ ]:
# Plot the number of combinations by subset size
plt.figure(figsize=(10, 6))
plt.bar(combinations_df['index'], combinations_df['Combinations'], color='skyblue', edgecolor='navy')

# Adding a curve to highlight the pattern
plt.plot(combinations_df['index'], combinations_df['Combinations'], 'ro-', linewidth=2)

# Add annotations for each point
for i, row in combinations_df.iterrows():
    plt.annotate(f"{int(row['Combinations'])}", 
                 (row['index'], row['Combinations']),
                 textcoords="offset points", 
                 xytext=(0,10), 
                 ha='center')

# Add labels and title
plt.xlabel('Size of Subset, |S|', fontsize=12)
plt.ylabel('Number of Combinations', fontsize=12)
plt.title(f'Number of Combinations for Subsets of |F| - 1 = 8 Features', fontsize=14)

# Format x-axis
plt.xticks(combinations_df['index'])
plt.grid(axis='y', linestyle='--', alpha=0.7)

# Add a note about the total
plt.figtext(0.5, 0.01, f"Total number of combinations: {total_combinations}", 
            ha="center", fontsize=12, bbox={"facecolor":"lightgray", "alpha":0.5, "pad":5})

plt.tight_layout(rect=[0, 0.03, 1, 0.95])
plt.show()

As you can see, the number of possible combinations explodes exponentially with the number of features. The number of subsets in $F/\{i\}$ is 254, and since there are 9 features, the number of subsets to evaluate is:

In [ ]:
len(X.columns) * total_combinations

That is a lot of subsets! As you can see, the number of subsets explodes exponentially with the number of features.


### Computational Difficulties and TreeSHAP

There are two main issues with SHAP values:

1. Estimating the conditional expectations $E[f(x) | x_S]$ efficiently
2. The combinatorical complexity of the SHAP value equation

Fortunately, the breakthrough that Lundberg, Erion and Lee made in 2019 [2]  discovering a fast algorithm for computing SHAP values for tree-based models. The model is polynomial time, and allows large models with many features on large dataset to be explained quickly using SHAP. The algorithm is called TreeSHAP. 

The algorithm is able to compute  $E[f(x) | x_S]$ and does not require sampling or assuming features are independent (more on this later).

# Visualizations and Applications

# References

1. Lundberg, S. M., & Lee, S.-I. (2017). *A Unified Approach to Interpreting Model Predictions*. In Proceedings of the 31st International Conference on Neural Information Processing Systems (NIPS 2017). [https://arxiv.org/abs/1705.07874](https://arxiv.org/abs/1705.07874)

2. Lundberg, S. M., Erion, G. G., & Lee, S.-I. (2019). *Consistent Individualized Feature Attribution for Tree Ensembles*. arXiv preprint arXiv:1802.03888. [https://arxiv.org/abs/1802.03888](https://arxiv.org/abs/1802.03888)

3. Frye, C., Rowat, C., & Feige, I. (2020). *Asymmetric Shapley Values: Incorporating Causal Knowledge into Model-Agnostic Explainability*. In Advances in Neural Information Processing Systems (NeurIPS 2020). https://arxiv.org/abs/1910.06358

4. SHAP Documentation: https://shap.readthedocs.io

5. Molnar, Christoph. *Interpretable Machine Learning: A Guide for Making Black Box Models Explainable*. 3rd ed., 2025. ISBN: 978-3-911578-03-5. Available at: https://christophm.github.io/interpretable-ml-book


